In [21]:
from selenium import webdriver
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style

print(plt.style.available)
%matplotlib inline

driver=webdriver.Edge() #打开浏览器
url='https://tj.esf.fang.com/house-a037-b0154/' #房源链接
driver.get(url)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [22]:
string_list=[]
i=0
#数据获取
while i<20:
    table = driver.find_element(By.CLASS_NAME,'shop_list.shop_list_4')
    # Extract table rows
    rows = table.find_elements(By.TAG_NAME, 'dl')
    # Initialize lists to store data
    data = []
    # Loop through rows and extract data
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'dd')
        if cells:  # Skip rows without data (e.g., headers)
            row_data = [cell.text for cell in cells]
            data.append(row_data)
    df = pd.DataFrame(data)
    string_list.append(df)
    #driver.refresh()
    try:
        next_page=driver.find_element(By.LINK_TEXT,'下一页')
        next_page.click()
        i+=1
    except NoSuchElementException:
        break


In [ ]:
def split_info(string_list,index_num):
    info = string_list[0][0][index_num].split("\n")
    price_info = string_list[0][1][index_num].split("\n")
    #数据清洗
    if info[1]=="双重验真":#异常值处理
        title = info[0]
        address = info[3]
        price = price_info[0]
        unit_price = price_info[1]
        type =  info[2].split("|")[0]
        square =  info[2].split("|")[1]
        floor =  info[2].split("|")[2]
        direction =  info[2].split("|")[3]
        district =  info[4]
        traffic =  info[-1]
        if len(info[2].split("|")) ==6:#异常值处理
            date = info[1].split("|")[4]
        else:
            date = None
    else:
        title = info[0]
        address = info[2]
        price = price_info[0]
        unit_price = price_info[1]
        type =  info[1].split("|")[0]
        square =  info[1].split("|")[1]
        floor =  info[1].split("|")[2]
        direction =  info[1].split("|")[3]
        district =  info[3]
        traffic =  info[-1]
        if len(info[1].split("|")) ==6:#异常值处理
            date = info[1].split("|")[4]
        else:
            date = None
    return [title,address,price,unit_price,type,square,floor,direction,date,district,traffic]


In [ ]:
df = pd.DataFrame(columns=["标题","小区","总价","单价","户型","面积","层型","朝向","建成时间","区域","交通信息"])
for i in range(len(string_list)):#遍历子列表 写入数据
    try:
        for j in range(len(string_list[i])):
            if j>59:
                i+=1
            infomation = split_info(string_list,j)
            new_row = pd.DataFrame([infomation], columns=df.columns)
            df = pd.concat([df, new_row], ignore_index=True)
    except Exception as e:
        print(f"处理第 {i+1} 个子列表的第{j+1}条数据时出错: {e}")

In [ ]:
df


In [ ]:
df.to_excel("queyechang_esf_data.xlsx")